In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from utils.basic_fuctions import preprocess_features
from utils.AL_utils import basic_active_learning
import keel_ds
from BasicActiveLearning import ActiveLearningPipeline

# Load Data

In [2]:
k = keel_ds.list_data()
imbalanced_datasets_names = k[-1] # Imbalanced datasets

In [3]:
imbalanced_datasets_proper = []
for dataset in imbalanced_datasets_names:
  missing_datasets = ['vowel.npz', 'breast.npz', 'housevotes.npz', 'bupa.npz', 'zoo-3.npz', 'abalone9-18.npz', 'ecoli-0-3-4_vs_5.npz', 'glass4', 'lymphography-normal-fibrosis.npz', 'ecoli-0-1-3-7_vs_2-6.npz', 'shuttle-c2-vs-c4.npz', 'winequality-white-9_vs_4.npz'] # Wrong dataset name in imbalance set (error in library)
  if dataset in missing_datasets:
    continue
  dataset = dataset.replace('.npz', '')
  imbalanced_datasets_proper.append(dataset)
  df = keel_ds.load_data(dataset, imbalanced=True, raw=True)

# Active Learning pipeline

In [4]:
final_results_file = "final_results.csv"

for dataset in imbalanced_datasets_proper:

    final_results = []

    df = keel_ds.load_data(dataset, imbalanced=True, raw=True)
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1].str.strip().map({'negative': 0, 'positive': 1})

    skf = StratifiedKFold(n_splits=5)

    for train_index, test_index in skf.split(X, y):
        x_train_fold, x_test_fold = X.iloc[train_index], X.iloc[test_index]
        y_train_fold, y_test_fold = y.iloc[train_index], y.iloc[test_index]

        # Preporcess features for model
        x_train_fold = preprocess_features(x_train_fold)
        x_test_fold = preprocess_features(x_test_fold)

        pipeline = ActiveLearningPipeline(
                model=DecisionTreeClassifier(criterion='gini', max_depth=100),
                model_name='decision_tree',
                #model = KNeighborsClassifier(),
                #model_name ='knn',
                budget=50,
                cycle_budget=10,
                samples_selection_metric='margin_sampling'
            )

        metrics_file_name = pipeline.run_active_learning(
                X_train=x_train_fold,
                y_train=y_train_fold,
                X_test=x_test_fold,
                y_test=y_test_fold,
                dataset_name=dataset
            )

    pipeline.process_results(metrics_file_name, dataset, final_results)

    # Append results
    final_results_file = "final_results.csv"
    results_df = pd.DataFrame(final_results)

    if os.path.exists(final_results_file):
        df_from_file = pd.read_csv(final_results_file)
        final_results_df = pd.concat([df_from_file, results_df])
    else:
        final_results_df = results_df


    final_results_df.to_csv("final_results.csv", index=False)

1069


UnboundLocalError: local variable 'df_with_proba' referenced before assignment

In [ ]:
import os
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import StratifiedKFold

# Define parameters
acquisition_functions = ['margin_sampling', 'entropy', 'least_confidence']
models = {
    'decision_tree': DecisionTreeClassifier(criterion='gini', max_depth=100),
    'KNN': KNeighborsClassifier(n_neighbors=5)
}
BUDGETS = [30, 40, 60, 70]
final_results_file = "final_results.csv"

# Iterate over datasets
for dataset in imbalanced_datasets_proper:
    # Store results for the dataset
    final_results = []
    
    # Load dataset
    df = keel_ds.load_data(dataset, imbalanced=True, raw=True)
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1].str.strip().map({'negative': 0, 'positive': 1})

    # Stratified K-Fold
    skf = StratifiedKFold(n_splits=5)

    for train_index, test_index in skf.split(X, y):
        x_train_fold, x_test_fold = X.iloc[train_index], X.iloc[test_index]
        y_train_fold, y_test_fold = y.iloc[train_index], y.iloc[test_index]

        # Preprocess features
        x_train_fold = preprocess_features(x_train_fold)
        x_test_fold = preprocess_features(x_test_fold)

        # Iterate over models
        for model_name, model in models.items():
            # Iterate over budgets
            for budget in BUDGETS:
                # Iterate over acquisition functions
                for acquisition_function in acquisition_functions:
                    # Initialize pipeline
                    pipeline = ActiveLearningPipeline(
                        model=model,
                        model_name=model_name,
                        budget=budget,
                        cycle_budget=10,  # Assuming fixed cycle budget
                        samples_selection_metric=acquisition_function
                    )
                    
                    # Run active learning and get metrics file
                    metrics_file_name = pipeline.run_active_learning(
                        X_train=x_train_fold,
                        y_train=y_train_fold,
                        X_test=x_test_fold,
                        y_test=y_test_fold,
                        dataset_name=dataset
                    )
                    
                    # Process results
                    pipeline.process_results(metrics_file_name, dataset, final_results)

    # Save results for the dataset
    results_df = pd.DataFrame(final_results)
    
    # Append results to the final CSV file
    if os.path.exists(final_results_file):
        df_from_file = pd.read_csv(final_results_file)
        final_results_df = pd.concat([df_from_file, results_df], ignore_index=True)
    else:
        final_results_df = results_df

    final_results_df.to_csv(final_results_file, index=False)


1069
(1069, 8)
[1 0]


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

# Active learning flow

In [8]:
acquistion_functions = ['margin_sampling', 'entropy', 'least_confidence']
models = {'decision_tree': DecisionTreeClassifier(criterion='gini', max_depth=100), 'KNN': KNeighborsClassifier(n_neighbors=5)}
BUDGETS = [30, 40, 60, 70]

In [5]:
final_results = []

for dataset in imbalanced_datasets_proper:
    print(dataset)
    df = keel_ds.load_data(dataset, imbalanced=True, raw=True)

    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]

    y = y.str.strip()
    y = y.map({'negative': 0, 'positive': 1})

    model = DecisionTreeClassifier(criterion='gini', max_depth=100)
    model_name = 'decision_tree'
    dataset_name = dataset  # Assuming `dataset` is predefined
    budget = 50  # Assuming budget value
    cycle_budget=10
    samples_selection_metric = 'entropy'

    # Prepare data for cross validation
    skf = StratifiedKFold(n_splits=5)
    for train_index, test_index in skf.split(X, y):
        x_train_fold, x_test_fold = X.iloc[train_index], X.iloc[test_index]
        y_train_fold, y_test_fold = y.iloc[train_index], y.iloc[test_index]

        # Preporcess features for model
        x_train_fold = preprocess_features(x_train_fold)
        x_test_fold = preprocess_features(x_test_fold)


        model = DecisionTreeClassifier(criterion='gini', max_depth=100)
        model_name = 'decisin_tree'
        metrics_file_name = basic_active_learning(x_train_fold, y_train_fold, budget, cycle_budget, model = model, model_name= model_name, dataset_name=dataset, X_test=x_test_fold, y_test=y_test_fold, samples_selection_metric=samples_selection_metric)
        
    results_df = pd.read_csv(os.path.join('results', metrics_file_name))
    max_cycle = results_df['Cycle'].max()  # Find the maximum cycle
    last_cycle_df = results_df[results_df['Cycle'] == max_cycle]  # Filter for the last cycle

    mean_last_cycle = last_cycle_df.mean(numeric_only=True)
    std_last_cycle = last_cycle_df.std(numeric_only=True)
    mean_last_cycle.to_csv(f"Metrices_summaries/{metrics_file_name.replace('results', '')}__mean.csv")
    std_last_cycle.to_csv(f"Metrices_summaries/{metrics_file_name.replace('results', '')}__std.csv")
    
    # Save final results
    final_results.append({
            "dataset_name": dataset_name,
            "model_name": model_name,
            "mean_last_cycle": mean_last_cycle.to_dict(),
            "std_last_cycle": std_last_cycle.to_dict(),
            "number_of_cycles": max_cycle,
            "budget": budget
        })

# Save all final results to a file
final_results_df = pd.DataFrame(final_results)
final_results_df.to_csv("final_results.csv", index=False)

yeast3
[0 1]
Metrics saved for cycle 0 to results/decisin_tree__entropy__yeast3__classic_AL.csv

Overall Accuracy: 0.9192


ValueError: Input y contains NaN.

In [12]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split


def select_lowest_probabieties(probabilities, df, n_samples, labels):
    """
    Parametres:
     - probabilities: samples probabilities
     - df: DataFrame 
     - n_samples: number of samples
    Return:
     - df_top: DataFrame with samples with the lowest probabilities
     - df_rest: DataFrame contianing rest of samples
    """
    df_with_proba = df.copy()
    df_with_proba['probability'] = None
    df_with_proba['labels'] = None

    df_with_proba['probability'] = probabilities
    df_with_proba['labels'] = labels
    df_with_proba = df_with_proba.sort_values(by='probability', ascending=True)
    df_top = df_with_proba[:n_samples]
    X_top = df_top.drop(columns=['probability', 'labels'])
    y_top = df_top['labels']
    df_rest = df_with_proba[n_samples:]
    X_rest = df_rest.drop(columns=['probability', 'labels'])
    y_rest = df_rest['labels']

    return X_top, y_top, X_rest, y_rest

def save_metrics_to_file(folder_name, filename, cycle, precision, recall, f1, cm, accuracy, X_labelled, y_test, positive_class):
    """
    Save metrics only for the positive class to a CSV file.

    Parameters:
        folder_name (str): Name of the folder to save the file.
        filename (str): Name of the file to save the metrics.
        cycle (int): Cycle number.
        precision (list): Precision scores for each class.
        recall (list): Recall scores for each class.
        f1 (list): F1 scores for each class.
        cm (ndarray): Confusion matrix.
        accuracy (float): Overall accuracy.
        X_labelled (ndarray): Labelled dataset.
        y_test (ndarray): Ground truth labels.
        positive_class: The value of the positive class label.
    """
    # Ensure the folder exists, create it if necessary
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
    
    # Construct the full path to the file in the specified folder
    full_path = os.path.join(folder_name, filename)
    
    # Create a list to hold the results for this cycle
    results = []
    
    # Get the unique classes
    unique_classes = np.unique(y_test)
    
    # Check if the positive class exists
    if positive_class in unique_classes:
        # Get the index of the positive class
        positive_class_index = np.where(unique_classes == positive_class)[0][0]
        
        # Append results only for the positive class
        results.append({
            'Cycle': cycle,
            'Class': positive_class,
            'Precision': precision[positive_class_index],
            'Recall': recall[positive_class_index],
            'F1-score': f1[positive_class_index],
            'Accuracy': accuracy,
            'Confusion Matrix': cm.tolist(),
            'Labelled Samples': X_labelled.shape[0]
        })
    
    # Convert to DataFrame and save to CSV
    if results:  # Only save if there are results for the positive class
        results_df = pd.DataFrame(results)
        # Append to the CSV file (if it exists) or create a new one
        results_df.to_csv(full_path, mode='a', header=not pd.io.common.file_exists(full_path), index=False)
        print(f"Metrics saved for cycle {cycle} to {full_path}")
    else:
        print(f"No metrics to save for the positive class '{positive_class}' in cycle {cycle}.")

    
# Function to calculate metrics
def calculate_metrics(y_test, y_pred):
    precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average=None)
    cm = confusion_matrix(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    
    return precision, recall, f1, cm, accuracy

"""def select_samples(probalility, df, n_samples, labels, metric="least_confidence"):
    print(np.unique(labels))
    # Check if only one class is present
    if len(np.unique(labels)) == 1:
        print("Only one class present. Selecting samples randomly.")
        # Randomly sample `n_samples` from the data
        df_sampled = df_with_proba.sample(n=n_samples, random_state=42)
        df_rest = df_with_proba.drop(df_sampled.index)

        X_top = df_sampled.drop(columns=['labels'])
        y_top = df_sampled['labels']

        X_rest = df_rest.drop(columns=['labels'])
        y_rest = df_rest['labels']

        return X_top, y_top, X_rest, y_rest
    
    # Create a DataFrame to store the probabilities and labels
    df_with_proba = df.copy()
    df_with_proba['labels'] = labels

    # Compute the uncertainty scores based on the chosen metric
    if metric == "least_confidence":
        # Least Confidence: Select the samples with the lowest top predicted probabilities
        prob_top = np.max(probalility, axis=1)  # Get the top probability for each sample
        df_with_proba['uncertainty'] = prob_top

    elif metric == "entropy":
        # Entropy: Calculate the entropy for each sample based on its class probabilities
        entropy_values = -np.sum(probalility * np.log(probalility + 1e-10), axis=1)  # Adding small epsilon to avoid log(0)
        df_with_proba['uncertainty'] = entropy_values

    elif metric == "margin_sampling":
        # Margin Sampling: Select the samples where the top two predicted probabilities are closest
        top_2_probs = np.partition(probalility, -2, axis=1)[:, -2:]  # Get the top 2 predicted probabilities
        margin = top_2_probs[:, 1] - top_2_probs[:, 0]  # Calculate the difference between the top two
        df_with_proba['uncertainty'] = margin
        
    else:
        raise ValueError(f"Unknown metric: {metric}. Available options are 'least_confidence', 'entropy', 'margin_sampling'.")
    
    # Sort the samples by the uncertainty (ascending order to get the most uncertain samples)
    df_with_proba = df_with_proba.sort_values(by='uncertainty', ascending=True)
    
    # Select the top `n_samples` samples with the highest uncertainty
    df_top = df_with_proba[:n_samples]
    X_top = df_top.drop(columns=['uncertainty', 'labels'])
    y_top = df_top['labels']
    
    # Select the remaining samples
    df_rest = df_with_proba[n_samples:]
    X_rest = df_rest.drop(columns=['uncertainty', 'labels'])
    y_rest = df_rest['labels']
    
    return X_top, y_top, X_rest, y_rest"""

import numpy as np

def select_samples(probalility, df, n_samples, labels, metric="least_confidence"):
    """
    Select samples for active learning based on different uncertainty metrics.
    If the uncertainty scores are uniform, select random samples instead.
    """

    # Create a DataFrame to store the probabilities and labels
    df_with_proba = df.copy()
    df_with_proba['labels'] = labels.values
    print('Labell', labels.unique())
    print(len(labels), df_with_proba.shape)
    print(df_with_proba['labels'].unique())
    # Check if only one class is present
    # if len(np.unique(labels)) == 1:
    #     print("Only one class present. Selecting samples randomly.")
    #     # Randomly sample `n_samples` from the data
    #     df_sampled = df_with_proba.sample(n=n_samples)
    #     df_rest = df_with_proba.drop(df_sampled.index)

    #     X_top = df_sampled.drop(columns=['labels'])
    #     y_top = df_sampled['labels']

    #     X_rest = df_rest.drop(columns=['labels'])
    #     y_rest = df_rest['labels']

    #     return X_top, y_top, X_rest, y_rest

    # Compute the uncertainty scores based on the chosen metric
    if metric == "least_confidence":
        prob_top = np.max(probalility, axis=1)
        df_with_proba['uncertainty'] = 1 - prob_top

    elif metric == "entropy":
        entropy_values = -np.sum(probalility * np.log(probalility + 1e-10), axis=1)
        df_with_proba['uncertainty'] = entropy_values

    elif metric == "margin_sampling":
        top_2_probs = np.partition(probalility, -2, axis=1)[:, -2:]  # Get the top 2 predicted probabilities
        margin = top_2_probs[:, 1] - top_2_probs[:, 0]  # Calculate the difference between the top two
        df_with_proba['uncertainty'] = margin

    else:
        raise ValueError(f"Unknown metric: {metric}. Available options are 'least_confidence', 'entropy', 'margin_sampling'.")

    # Sort the samples by uncertainty (ascending order to get the most uncertain samples)
    df_with_proba = df_with_proba.sort_values(by='uncertainty', ascending=True)
    print(df_with_proba)

    # # If all uncertainty scores are the same, select random samples
    # if df_with_proba['uncertainty'].nunique() == 1:
    #     df_sampled = df_with_proba.sample(n=n_samples, random_state=42)
    #     df_rest = df_with_proba.drop(df_sampled.index)
    # else:
        # Select the top `n_samples` samples with the highest uncertainty
    df_sampled = df_with_proba[:n_samples]
    df_rest = df_with_proba[n_samples:]
    print(df_rest)
    print(df_rest)

    X_top = df_sampled.drop(columns=['uncertainty', 'labels'])
    y_top = df_sampled['labels']

    X_rest = df_rest.drop(columns=['uncertainty', 'labels'])
    y_rest = df_rest['labels']

    return X_top, y_top, X_rest, y_rest



def calculate_dynamic_class_weights_based_on_model(model, X_labelled, y_labeled):
    """
    Calculate dynamic class weights based on the performance of the model on the current labelled data.
    """
    y_pred = model.predict(X_labelled)
    
    # Find misclassifications (or measure uncertainty)
    misclassifications = (y_pred != y_labeled)
    
    # Calculate the frequency of misclassifications per class
    class_misclassifications = {class_label: np.sum(misclassifications[y_labeled == class_label]) 
                                for class_label in np.unique(y_labeled)}
    
    # Compute class weights as the inverse of misclassifications (more misclassified = higher weight)
    total_misclassifications = sum(class_misclassifications.values())
    class_weights = {class_label: (total_misclassifications / (class_misclassifications[class_label] + 1)) 
                     for class_label in class_misclassifications}
    
    return class_weights

def select_samples_weighted(probalility, df, n_samples, labels, metric, class_weights):
    """
    Select samples for active learning based on different uncertainty metrics and dynamically calculated class weights.

    Parameters:
    - probalility: Array of probabilities for each sample (shape: [n_samples, n_classes]).
    - df: The DataFrame containing the feature data.
    - n_samples: The number of samples to select.
    - labels: The true labels for the unlabelled samples.
    - metric: The uncertainty metric to use. Options are 'least_confidence', 'entropy', or 'margin_sampling'.
    - calculate_class_weights_fn: A function to dynamically calculate class weights.

    Returns:
    - X_top: The feature data for the selected samples.
    - y_top: The labels for the selected samples.
    - X_rest: The feature data for the remaining samples.
    - y_rest: The labels for the remaining samples.
    """
    
    # Create a DataFrame to store the probabilities and labels
    df_with_proba = df.copy()
    df_with_proba['labels'] = labels

    print(np.unique(labels))
    # Check if only one class is present
    if len(np.unique(labels)) == 1:
        print("Only one class present. Selecting samples randomly.")
        # Randomly sample `n_samples` from the data
        df_sampled = df_with_proba.sample(n=n_samples, random_state=42)
        df_rest = df_with_proba.drop(df_sampled.index)

        X_top = df_sampled.drop(columns=['labels'])
        y_top = df_sampled['labels']

        X_rest = df_rest.drop(columns=['labels'])
        y_rest = df_rest['labels']

        return X_top, y_top, X_rest, y_rest

    
    # Compute the uncertainty scores based on the chosen metric
    if metric == "least_confidence":
        # Least Confidence: Select the samples with the lowest top predicted probabilities
        prob_top = np.max(probalility, axis=1)  # Get the top probability for each sample
        df_with_proba['uncertainty'] = prob_top

    elif metric == "entropy":
        entropy_values = -np.sum(probalility * np.log(probalility + 1e-10), axis=1)  # Adding small epsilon to avoid log(0)
        df_with_proba['uncertainty'] = entropy_values

    elif metric == "margin_sampling":
        top_2_probs = np.partition(probalility, -2, axis=1)[:, -2:]  # Get the top 2 predicted probabilities
        margin = top_2_probs[:, 1] - top_2_probs[:, 0]  # Calculate the difference between the top two
        df_with_proba['uncertainty'] = margin
    else:
        raise ValueError(f"Unknown metric: {metric}. Available options are 'least_confidence', 'entropy', 'margin_sampling'.")

    # Apply class weighting to the uncertainty values if class weights are provided
    if class_weights:
        df_with_proba['uncertainty'] *= df_with_proba['labels'].map(class_weights)
    
    # Sort the samples by the uncertainty (ascending order to get the most uncertain samples)
    df_with_proba = df_with_proba.sort_values(by='uncertainty', ascending=True)
    
    # Select the top n_samples samples with the highest uncertainty
    df_top = df_with_proba[:n_samples]
    X_top = df_top.drop(columns=['uncertainty', 'labels'])
    y_top = df_top['labels']
    
    # Select the remaining samples
    df_rest = df_with_proba[n_samples:]
    X_rest = df_rest.drop(columns=['uncertainty', 'labels'])
    y_rest = df_rest['labels']
    
    return X_top, y_top, X_rest, y_rest


def basic_active_learning(X_train, y_train, budget, cycle_budget, model, model_name, dataset_name, X_test, y_test, samples_selection_metric='margin_sampling', test_size=0.9):
 
    # Split data to DL and DU
    X_labelled, X_unlabelled, y_labeled, y_unlabelled = train_test_split(X_train, y_train, test_size=test_size, stratify=y_train)

    # Whole budget
    B = budget
    # Budget per cycle
    b = cycle_budget
    # Number of cycle
    c = 0

    results_folder = 'results'

    while B>0:
        print(y_labeled.unique())
        model.fit(X_labelled, y_labeled)
        probalilities = model.predict_proba(X_unlabelled)
        print('kkkk', y_unlabelled.unique())

        # 2. Select samples based on choosen metrics and ask Oracle
        X_lowest_prob, y_lowest_proba, X_rest, y_rest = select_samples(probalility=probalilities, df=X_unlabelled, n_samples=b, labels=y_unlabelled, metric=samples_selection_metric)
        print(y_lowest_proba.unique())
        print(y_rest.unique())
        # 3. Add samples labelled by Oracle to DL
        X_labelled = pd.concat([X_labelled, X_lowest_prob])
        y_labeled = pd.concat([y_labeled, y_lowest_proba])
        # 4. Update DUL
        X_unlabelled = X_rest
        y_unlabelled = y_rest
        print(y_labeled.unique())
        print(y_lowest_proba.unique())



        # Calculate accuracy for this cycle
        y_pred = model.predict(X_test)

        metrics_filename = f'{model_name}__{samples_selection_metric}__{dataset_name}__classic_AL.csv'

        # Calculate metrics
        precision, recall, f1, cm, accuracy = calculate_metrics(y_test, y_pred)
      
        # Save the metrics to the file
        save_metrics_to_file(results_folder, metrics_filename, c, precision, recall, f1, cm, accuracy, X_labelled, y_test, positive_class=1)
          
        # Print accurracy
        accuracy = accuracy_score(y_test, y_pred)
        print(f"\nOverall Accuracy: {accuracy:.4f}")
      
        # Update cycle and budget
        c +=1
        B -=b
    return metrics_filename


# Class

In [56]:
import os
import numpy as np
import pandas as pd
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

class ActiveLearningPipeline:
    def __init__(self, model, model_name, budget, cycle_budget, samples_selection_metric, results_folder='results'):
        self.model = model
        self.model_name = model_name
        self.budget = budget
        self.cycle_budget = cycle_budget
        self.samples_selection_metric = samples_selection_metric
        self.results_folder = results_folder

        if not os.path.exists(self.results_folder):
            os.makedirs(self.results_folder)

    def load_data(self, folder_name, file_name, header=None):
        """Load csv file in df"""
        df = pd.read_csv(os.path.join(os.getcwd(), folder_name, file_name), header=header)
        return df

    def preprocess_features(self, X_df):
        X_df_copy = X_df.copy()
        # Convert categorical columns to numerical using LabelEncoder
        for col in X_df_copy.columns:
            if X_df_copy.loc[:, col].dtype == 'object':  # Check if the column is categorical
                le = LabelEncoder()
                X_df_copy.loc[:, col] = le.fit_transform(X_df_copy[col])

        # Now normalize the dataframe
        scaler = MinMaxScaler()
        X_normalized = scaler.fit_transform(X_df_copy)

        X_normalized = pd.DataFrame(X_normalized, columns=X_df_copy.columns)

        # Check the resulting DataFrame
        return X_normalized

    def select_samples(self, probabilities, df, n_samples, labels):
        df_with_proba = df.copy()
        df_with_proba['labels'] = labels.values

        if self.samples_selection_metric == "least_confidence":
            prob_top = np.max(probabilities, axis=1)
            df_with_proba['uncertainty'] = 1 - prob_top
        elif self.samples_selection_metric == "entropy":
            entropy_values = -np.sum(probabilities * np.log(probabilities + 1e-10), axis=1)
            df_with_proba['uncertainty'] = entropy_values
        elif self.samples_selection_metric == "margin_sampling":
            top_2_probs = np.partition(probabilities, -2, axis=1)[:, -2:]
            margin = top_2_probs[:, 1] - top_2_probs[:, 0]
            df_with_proba['uncertainty'] = margin
        else:
            raise ValueError(f"Unknown metric: {self.samples_selection_metric}")

        df_with_proba = df_with_proba.sort_values(by='uncertainty', ascending=True)
        df_sampled = df_with_proba[:n_samples]
        df_rest = df_with_proba[n_samples:]

        X_top = df_sampled.drop(columns=['uncertainty', 'labels'])
        y_top = df_sampled['labels']

        X_rest = df_rest.drop(columns=['uncertainty', 'labels'])
        y_rest = df_rest['labels']

        return X_top, y_top, X_rest, y_rest

    def calculate_metrics(self, y_test, y_pred):
        precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average=None)
        cm = confusion_matrix(y_test, y_pred)
        accuracy = accuracy_score(y_test, y_pred)
        return precision, recall, f1, cm, accuracy

    def save_metrics_to_file(self, filename, cycle, precision, recall, f1, cm, accuracy, X_labelled, y_test, positive_class):
        full_path = os.path.join(self.results_folder, filename)
        results = []
        unique_classes = np.unique(y_test)

        if positive_class in unique_classes:
            positive_class_index = np.where(unique_classes == positive_class)[0][0]
            results.append({
                'Cycle': cycle,
                'Class': positive_class,
                'Precision': precision[positive_class_index],
                'Recall': recall[positive_class_index],
                'F1-score': f1[positive_class_index],
                'Accuracy': accuracy,
                'Confusion Matrix': cm.tolist(),
                'Labelled Samples': X_labelled.shape[0]
            })

        if results:
            results_df = pd.DataFrame(results)
            results_df.to_csv(full_path, mode='a', header=not os.path.exists(full_path), index=False)
            # print(f"Metrics saved for cycle {cycle} to {full_path}")

    def run_active_learning(self, X_train, y_train, X_test, y_test, dataset_name):
        X_labelled, X_unlabelled, y_labelled, y_unlabelled = train_test_split(
            X_train, y_train, test_size=0.9, stratify=y_train
        )

        remaining_budget = self.budget
        cycle = 0

        while remaining_budget > 0:
            self.model.fit(X_labelled, y_labelled)
            probabilities = self.model.predict_proba(X_unlabelled)

            X_top, y_top, X_rest, y_rest = self.select_samples(probabilities, X_unlabelled, self.cycle_budget, y_unlabelled)

            X_labelled = pd.concat([X_labelled, X_top])
            y_labelled = pd.concat([y_labelled, y_top])
            X_unlabelled = X_rest
            y_unlabelled = y_rest

            y_pred = self.model.predict(X_test)
            precision, recall, f1, cm, accuracy = self.calculate_metrics(y_test, y_pred)

            metrics_filename = f'{self.model_name}__{self.samples_selection_metric}__{dataset_name}__classic_AL.csv'
            self.save_metrics_to_file(metrics_filename, cycle, precision, recall, f1, cm, accuracy, X_labelled, y_test, positive_class=1)

            # print(f"Cycle {cycle} Accuracy: {accuracy:.4f}")

            cycle += 1
            remaining_budget -= self.cycle_budget

        return metrics_filename

    def process_results(self, metrics_file_name, dataset, final_results):
        results_df = pd.read_csv(os.path.join('results', metrics_file_name))
        print(results_df)

        if not os.path.exists('Metrics_summaries'):
            os.makedirs('Metrics_summaries')
        
        last_iteration_number = max(results_df['Cycle'])
        grouped_df = results_df.groupby('Cycle').agg(['mean', 'std'])
        
        grouped_df.columns = ['_'.join(col).strip() for col in grouped_df.columns.values]
        last_iter_stats = grouped_df.loc[last_iteration_number, :]
        stats_dict = last_iter_stats.to_dict()  # Convert Series to dictionary
        print(stats_dict)

        # # Append results to final summary
        final_results.append({
             "dataset_name": dataset,
             "model_name": self.model_name,
             "mean_and_stats_for_last_iter": stats_dict,
             "cycle": last_iteration_number,
             "budget": self.budget
         })

if __name__ == "__main__":
    final_results = []
    #imbalanced_datasets_proper = ['abalone-3_vs_11']
    for dataset in imbalanced_datasets_proper:
        # print(f"Processing dataset: {dataset}")

        df = keel_ds.load_data(dataset, imbalanced=True, raw=True)
        X = df.iloc[:, :-1]
        y = df.iloc[:, -1].str.strip().map({'negative': 0, 'positive': 1})

        skf = StratifiedKFold(n_splits=5)

        for train_index, test_index in skf.split(X, y):
            x_train_fold, x_test_fold = X.iloc[train_index], X.iloc[test_index]
            y_train_fold, y_test_fold = y.iloc[train_index], y.iloc[test_index]

            # Preporcess features for model
            x_train_fold = preprocess_features(x_train_fold)
            x_test_fold = preprocess_features(x_test_fold)

            pipeline = ActiveLearningPipeline(
                model=DecisionTreeClassifier(criterion='gini', max_depth=100),
                model_name='decision_tree',
                budget=50,
                cycle_budget=10,
                samples_selection_metric='entropy'
            )

            metrics_file_name = pipeline.run_active_learning(
                X_train=x_train_fold,
                y_train=y_train_fold,
                X_test=x_test_fold,
                y_test=y_test_fold,
                dataset_name=dataset
            )

        pipeline.process_results(metrics_file_name, dataset, final_results)

    final_results_df = pd.DataFrame(final_results)
    final_results_df.to_csv("final_results.csv", index=False)


    Cycle  Class  Precision    Recall  F1-score  Accuracy  \
0       0      1   0.333333  0.156250  0.212766  0.875421   
1       1      1   0.333333  0.187500  0.240000  0.872054   
2       2      1   0.500000  0.187500  0.272727  0.892256   
3       3      1   0.473684  0.281250  0.352941  0.888889   
4       4      1   0.571429  0.250000  0.347826  0.898990   
5       0      1   0.800000  0.363636  0.500000  0.919192   
6       1      1   0.785714  0.333333  0.468085  0.915825   
7       2      1   0.782609  0.545455  0.642857  0.932660   
8       3      1   0.791667  0.575758  0.666667  0.936027   
9       4      1   0.642857  0.272727  0.382979  0.902357   
10      0      1   0.191011  0.515152  0.278689  0.703704   
11      1      1   0.375000  0.545455  0.444444  0.848485   
12      2      1   0.156627  0.393939  0.224138  0.696970   
13      3      1   0.271028  0.878788  0.414286  0.723906   
14      4      1   0.252252  0.848485  0.388889  0.703704   
15      0      1   0.615

/var/folders/bl/bmspcss576x1_nsjjqdhbjvw0000gn/T/ipykernel_34604/165928778.py:141: FutureWarning: ['Confusion Matrix'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  grouped_df = results_df.groupby('Cycle').agg(['mean', 'std'])
/var/folders/bl/bmspcss576x1_nsjjqdhbjvw0000gn/T/ipykernel_34604/165928778.py:141: FutureWarning: ['Confusion Matrix'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  grouped_df = results_df.groupby('Cycle').agg(['mean', 'std'])
/var/folders/bl/bmspcss576x1_nsjjqdhbjvw0000gn/T/ipykernel_34604/165928778.py:141: FutureWarning: ['Confusion Matrix'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  grouped_df = results_df.groupby('Cycle').agg(['mean', 'std'])


    Cycle  Class  Precision    Recall  F1-score  Accuracy  \
0       0      1   0.202000  0.901786  0.330065  0.625571   
1       1      1   0.206287  0.937500  0.338164  0.624658   
2       2      1   0.204365  0.919643  0.334416  0.625571   
3       3      1   0.203156  0.919643  0.332795  0.622831   
4       4      1   0.203187  0.910714  0.332248  0.625571   
5       0      1   0.095187  0.794643  0.170010  0.206393   
6       1      1   0.089013  0.571429  0.154031  0.357991   
7       2      1   0.096956  0.767857  0.172172  0.244749   
8       3      1   0.108961  0.955357  0.195612  0.196347   
9       4      1   0.156805  0.473214  0.235556  0.685845   
10      0      1   0.086729  0.747748  0.155431  0.175503   
11      1      1   0.098563  0.864865  0.176959  0.183729   
12      2      1   0.107692  0.945946  0.193370  0.199269   
13      3      1   0.108025  0.945946  0.193906  0.202011   
14      4      1   0.092037  0.801802  0.165121  0.177331   
15      0      1   0.114

/var/folders/bl/bmspcss576x1_nsjjqdhbjvw0000gn/T/ipykernel_34604/165928778.py:141: FutureWarning: ['Confusion Matrix'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  grouped_df = results_df.groupby('Cycle').agg(['mean', 'std'])
/Users/marysia/miniconda3/envs/myenv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/marysia/miniconda3/envs/myenv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu

    Cycle  Class  Precision    Recall  F1-score  Accuracy    Confusion Matrix  \
0       0      1   0.000000  0.000000  0.000000  0.471910  [[42, 40], [7, 0]]   
1       1      1   0.666667  0.857143  0.750000  0.955056   [[79, 3], [1, 6]]   
2       2      1   0.833333  0.714286  0.769231  0.966292   [[81, 1], [2, 5]]   
3       3      1   0.833333  0.714286  0.769231  0.966292   [[81, 1], [2, 5]]   
4       4      1   0.666667  0.857143  0.750000  0.955056   [[79, 3], [1, 6]]   
5       0      1   0.000000  0.000000  0.000000  0.910112   [[81, 0], [8, 0]]   
6       1      1   0.000000  0.000000  0.000000  0.910112   [[81, 0], [8, 0]]   
7       2      1   0.000000  0.000000  0.000000  0.910112   [[81, 0], [8, 0]]   
8       3      1   0.000000  0.000000  0.000000  0.910112   [[81, 0], [8, 0]]   
9       4      1   1.000000  0.125000  0.222222  0.921348   [[81, 0], [7, 1]]   
10      0      1   1.000000  1.000000  1.000000  1.000000   [[81, 0], [0, 8]]   
11      1      1   1.000000 

...

/var/folders/bl/bmspcss576x1_nsjjqdhbjvw0000gn/T/ipykernel_34604/165928778.py:141: FutureWarning: ['Confusion Matrix'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  grouped_df = results_df.groupby('Cycle').agg(['mean', 'std'])
/var/folders/bl/bmspcss576x1_nsjjqdhbjvw0000gn/T/ipykernel_34604/165928778.py:141: FutureWarning: ['Confusion Matrix'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  grouped_df = results_df.groupby('Cycle').agg(['mean', 'std'])
/Users/marysia/miniconda3/envs/myenv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/

    Cycle  Class  Precision    Recall  F1-score  Accuracy  \
0       0      1   0.107143  0.214286  0.142857  0.895954   
1       1      1   0.076923  0.142857  0.100000  0.895954   
2       2      1   0.096774  0.214286  0.133333  0.887283   
3       3      1   0.000000  0.000000  0.000000  0.861272   
4       4      1   0.000000  0.000000  0.000000  0.864162   
5       0      1   0.181818  0.142857  0.160000  0.939306   
6       1      1   0.000000  0.000000  0.000000  0.959538   
7       2      1   0.217391  0.357143  0.270270  0.921965   
8       3      1   0.333333  0.571429  0.421053  0.936416   
9       4      1   0.181818  0.285714  0.222222  0.919075   
10      0      1   0.000000  0.000000  0.000000  0.901734   
11      1      1   0.000000  0.000000  0.000000  0.901734   
12      2      1   0.000000  0.000000  0.000000  0.867052   
13      3      1   0.000000  0.000000  0.000000  0.867052   
14      4      1   0.000000  0.000000  0.000000  0.901734   
15      0      1   0.000

/Users/marysia/miniconda3/envs/myenv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/var/folders/bl/bmspcss576x1_nsjjqdhbjvw0000gn/T/ipykernel_34604/165928778.py:141: FutureWarning: ['Confusion Matrix'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  grouped_df = results_df.groupby('Cycle').agg(['mean', 'std'])
/Users/marysia/miniconda3/envs/myenv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu

    Cycle  Class  Precision  Recall  F1-score  Accuracy    Confusion Matrix  \
0       0      1   0.571429    0.80  0.666667  0.918367   [[41, 3], [1, 4]]   
1       1      1   0.666667    0.80  0.727273  0.938776   [[42, 2], [1, 4]]   
2       2      1   0.600000    0.60  0.600000  0.918367   [[42, 2], [2, 3]]   
3       3      1   0.666667    0.40  0.500000  0.918367   [[43, 1], [3, 2]]   
4       4      1   0.500000    0.40  0.444444  0.897959   [[42, 2], [3, 2]]   
5       0      1   0.333333    0.60  0.428571  0.836735   [[38, 6], [2, 3]]   
6       1      1   0.750000    0.60  0.666667  0.938776   [[43, 1], [2, 3]]   
7       2      1   0.250000    0.20  0.222222  0.857143   [[41, 3], [4, 1]]   
8       3      1   0.250000    0.20  0.222222  0.857143   [[41, 3], [4, 1]]   
9       4      1   0.571429    0.80  0.666667  0.918367   [[41, 3], [1, 4]]   
10      0      1   0.102041    1.00  0.185185  0.102041   [[0, 44], [0, 5]]   
11      1      1   0.102041    1.00  0.185185  0.102

/var/folders/bl/bmspcss576x1_nsjjqdhbjvw0000gn/T/ipykernel_34604/165928778.py:141: FutureWarning: ['Confusion Matrix'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  grouped_df = results_df.groupby('Cycle').agg(['mean', 'std'])
/Users/marysia/miniconda3/envs/myenv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/marysia/miniconda3/envs/myenv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu